In [50]:
from IPython.display import HTML

In [59]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('#code').text('Show code');
 $('#code').attr('class', 'btn btn-danger');
 } else {
 $('div.input').show();
 $('#code').text('Hide code');
 $('#code').attr('class', 'btn btn-success');
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <br><br>
<center>
<button style='height: 50px; width: 200px; font-size: 2em' id="code" type="button" onclick="javascript:code_toggle()">here</button>
</center>''')


# Dieselgate Routing

From march 2019 on, Berlin will impose the Diesel ban. The following streets will be affected: Leipziger Straße, Reinhardstraße, Friedrichstraße, Brückenstraße, Kapweg, Alt-Moabit, Stromstraße und Leonorenstraße. A recent [Spiegel](https://www.spiegel.de) comic illustrated the issue:

![title](http://cdn3.spiegel.de/images/image-1349732-galleryV9-ikww-1349732.jpg)
© [SPIEGEL](https://www.spiegel.de)

This notebook explores the validity of the comic's claim. It gives you the opportunity to check if and how your usual route will be affected and how an alternative route could look like. 

In [113]:
# Needed packages
import openrouteservice
from openrouteservice import geocode, client

import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, clear_output, HTML
#import functools
#import concurrent.futures
import asyncio
import time
import folium
from shapely.geometry import LineString, Polygon, mapping
from shapely.ops import cascaded_union


# Basic parameters
api_key = 'your_key' #https://openrouteservice.org/sign-up
clnt = client.Client(key='5b3ce3597851110001cf624870cf2f2a58d44c718542b3088221b684')

In [114]:
%gui asyncio # Needed for interacting tool

ERROR:root:Invalid GUI request 'asyncio # Needed for interacting tool', valid ones are:dict_keys(['inline', 'nbagg', 'notebook', 'ipympl', 'widget', None, 'qt4', 'qt', 'qt5', 'wx', 'tk', 'gtk', 'gtk3', 'osx', 'asyncio'])


In [115]:
# Waits for user to type in address
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

# Autocompletes the typed in address; Focus on Berlin
def autocomplete_address(input_text):
    poi_start = geocode.pelias_autocomplete(client=clnt, text=input_text.value, focus_point = [13.406582, 52.520765])
    return poi_start 

# To style data
def style_function(color): 
    return lambda feature: dict(color=color,
                                opacity=0.6,
                                weight=5,)

### Regular Route

Type in and chose your start and goal location and check out the shortest route which will be valid until march 2019. The map focus' on Berlin, but locations in other cities and countries can be chosen, too. Just give it try.

In [144]:
# Create interactive widgets
input_start = widgets.Text(description = "Start:")
choice_start = widgets.Select(
            options=[],
            layout=Layout(width="50%"))

input_dest = widgets.Text(description = "Destination:")
choice_dest = widgets.Select(
            options=[],
            layout=Layout(width="50%"))

try:
    # Search for start address
    async def text(): 
        for i in range(30):
            time.sleep(0.5)
            poi_list_start = []
            x = await wait_for_change(input_start, 'value')
            poi_start = autocomplete_address(input_start)
            for poi in poi_start['features']:
                poi_list_start += [(poi['properties']['label'], poi['geometry']['coordinates'])]
            choice_start.options=poi_list_start
    asyncio.ensure_future(text())     
    
    # Search for destination address
    async def text_dest():
        for j in range(30):
            poi_list_dest = []
            time.sleep(0.5)
            y = await wait_for_change(input_dest, 'value')
            poi_dest = autocomplete_address(input_dest)
            for poi in poi_dest['features']:
                poi_list_dest += [(poi['properties']['label'], poi['geometry']['coordinates'])]
            choice_dest.options=poi_list_dest
    asyncio.ensure_future(text_dest())   
    
except Exception:
    pass
    
display(input_start, choice_start)
display(input_dest, choice_dest)

Text(value='', description='Start:')

Select(layout=Layout(width='50%'), options=(), value=None)

Text(value='', description='Destination:')

Select(layout=Layout(width='50%'), options=(), value=None)

Click on the route, visualized on the map, to get information about the duration and distance.

In [145]:
choice_start.options

()

In [143]:
if choice_start.options or choice_dest.options == ():
    print('Start and destination are required.')
    print(choice_start.options)
        

Start and destination are required.
(('Paulstraße 28, Berlin, Germany', [13.356833, 52.522221]), ('Paulstraße 36, Berlin, Germany', [13.357195, 52.523222]), ('Paulstraße 35, Berlin, Germany', [13.357103, 52.523019]), ('Paulstraße 34, Berlin, Germany', [13.356869, 52.522893]), ('Paulstraße 27, Berlin, Germany', [13.356425, 52.521583]), ('Paulstraße, Berlin, Germany', [13.356738, 52.52321]), ('Paulstraße 26, Berlin, Germany', [13.356438, 52.521407]), ('Paulstraße 25, Berlin, Germany', [13.356389, 52.521263]), ('Paulstraße 1, Berlin, Germany', [13.3565, 52.523255]), ('Paulstraße 2, Berlin, Germany', [13.356465, 52.523115]))


In [125]:
if choice_start or choice_dest is None:
     print('Start and destination are required.')
#else:

#try:
# Create map of Berlin with chosen locations
map_berlin = folium.Map(tiles='https://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}', 
                        attr='Map data (c) OpenStreetMap, Tiles (c) <a href="https://heigit.org">GIScience Heidelberg</a>', 
                        location=([52.516586, 13.381047]), 
                        zoom_start=12) # Create map

folium.Marker(list(reversed(choice_start.value)), popup=choice_start.label).add_to(map_berlin)
folium.Marker(list(reversed(choice_dest.value)), popup=choice_dest.label).add_to(map_berlin)


# Regular Route
popup_route = "<h4>{0} route</h4><hr>" \
             "<strong>Duration: </strong>{1:.1f} mins<br>" \
             "<strong>Distance: </strong>{2:.3f} km" 

# Request route
direction_params = {'coordinates': [choice_start.value, choice_dest.value],
                    'profile': 'driving-car', 
                    'format_out': 'geojson',
                    'preference': 'shortest',
                    'optimized': True,
                    'geometry': 'true'}

regular_route = clnt.directions(**direction_params) # Direction request

# Build popup
duration, distance = regular_route['features'][0]['properties']['summary'][0].values()
popup = folium.Popup(popup_route.format('Regular', 
                                                 duration/60, 
                                                 distance/1000))

gj= folium.GeoJson(regular_route,
                   name='Regular Route',
                   style_function=style_function('blue')).add_child(popup).add_to(map_berlin)


# Start and destination coordinates of affectes streets
avoid_streets = [{'name':'Friedrichstraße', 'coords': [[13.38859, 52.518526], [13.388611, 52.518193]]}, 
                 {'name': 'Leipziger Straße', 'coords': [[13.381487, 52.509829], [13.383805, 52.509998]]},
                 {'name': 'Leipziger Straße', 'coords': [[13.387742, 52.51022], [13.389534, 52.510338]]},
                 {'name': 'Leipziger Straße', 'coords': [[13.39022, 52.510384], [13.391529, 52.510449]]},
                 {'name': 'Brückenstraße', 'coords': [[13.416549, 52.511193], [13.418105, 52.515209]]},
                 {'name': 'Kapweg', 'coords': [[13.326995, 52.56248], [13.330343, 52.562349]]},
                 {'name': 'Reinhardtstraße', 'coords': [[13.376777, 52.522098], [13.377131, 52.522162]]},
                 {'name': 'Reinhardtstraße', 'coords': [[13.377657, 52.522266], [13.379041, 52.522488]]},
                 {'name': 'Alt-Moabit', 'coords': [[13.32765, 52.524316], [13.329313, 52.523859]]}, 
                 {'name': 'Leonorenstraße', 'coords': [[13.344741, 52.437556], [13.345503, 52.43645]]},
                 {'name': 'Stromstraße', 'coords': [[13.34326, 52.527952], [13.343228, 52.526712]]}]


# Affected streets
buffer = []
for street in avoid_streets:
    avoid_params = {'coordinates': street['coords'],
                    'profile': 'driving-car', 
                    'format_out': 'geojson',
                    'preference': 'shortest',
                    'geometry': 'true'}
    avoid_request = clnt.directions(**avoid_params)
    coords = avoid_request['features'][0]['geometry']['coordinates']
    route_buffer = LineString(coords).buffer(0.0002) # Create geometry buffer
    folium.PolyLine([(y,x) for x,y in list(route_buffer.exterior.coords)],
                    color='#cc0000', fill_color='#cc0000',
                    popup=street['name']).add_to(map_berlin)
    simp_geom = route_buffer.simplify(0.0002) # Simplify geometry for better handling
    buffer.append(simp_geom)
union_buffer = cascaded_union(buffer)


# New routing with avoided streets
diesel_request = {'coordinates': [choice_start.value, choice_dest.value], 
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': False,
                 'options': {'avoid_polygons': mapping(union_buffer)}} 
route_diesel = clnt.directions(**diesel_request)

# Build popup
duration, distance = route_diesel['features'][0]['properties']['summary'][0].values()
popup = folium.Popup(popup_route.format('Diesel Route',
                                                 duration/60,
                                                 distance/1000))

folium.GeoJson(route_diesel,
               style_function=style_function('#009900'), 
               name='Route after Jan 2019').add_child(popup).add_to(map_berlin)


map_berlin.add_child(folium.map.LayerControl())
map_berlin  
    
# except Exception:
#     pass

Start and destination are required.


### Dieselgate Routing

Coming soon: The shortest route for a Diesel driver, which must avoid the reddish areas. Then, affected cars can't cross the designated streets anymore. But see for yourself:

In [ ]:
# Start and destination coordinates of affectes streets
avoid_streets = [{'name':'Friedrichstraße', 'coords': [[13.38859, 52.518526], [13.388611, 52.518193]]}, 
                 {'name': 'Leipziger Straße', 'coords': [[13.381487, 52.509829], [13.383805, 52.509998]]},
                 {'name': 'Leipziger Straße', 'coords': [[13.387742, 52.51022], [13.389534, 52.510338]]},
                 {'name': 'Leipziger Straße', 'coords': [[13.39022, 52.510384], [13.391529, 52.510449]]},
                 {'name': 'Brückenstraße', 'coords': [[13.416549, 52.511193], [13.418105, 52.515209]]},
                 {'name': 'Kapweg', 'coords': [[13.326995, 52.56248], [13.330343, 52.562349]]},
                 {'name': 'Reinhardtstraße', 'coords': [[13.376777, 52.522098], [13.377131, 52.522162]]},
                 {'name': 'Reinhardtstraße', 'coords': [[13.377657, 52.522266], [13.379041, 52.522488]]},
                 {'name': 'Alt-Moabit', 'coords': [[13.32765, 52.524316], [13.329313, 52.523859]]}, 
                 {'name': 'Leonorenstraße', 'coords': [[13.344741, 52.437556], [13.345503, 52.43645]]},
                 {'name': 'Stromstraße', 'coords': [[13.34326, 52.527952], [13.343228, 52.526712]]}]


# Affected streets
buffer = []
for street in avoid_streets:
    avoid_params = {'coordinates': street['coords'],
                    'profile': 'driving-car', 
                    'format_out': 'geojson',
                    'preference': 'shortest',
                    'geometry': 'true'}
    avoid_request = clnt.directions(**avoid_params)
    coords = avoid_request['features'][0]['geometry']['coordinates']
    route_buffer = LineString(coords).buffer(0.0002) # Create geometry buffer
    folium.PolyLine([(y,x) for x,y in list(route_buffer.exterior.coords)],
                    color='#cc0000', fill_color='#cc0000',
                    popup=street['name']).add_to(map_berlin)
    simp_geom = route_buffer.simplify(0.0002) # Simplify geometry for better handling
    buffer.append(simp_geom)
union_buffer = cascaded_union(buffer)
map_berlin

The shorted route from A to B will be then be like this:

In [ ]:
# New routing with avoided streets
diesel_request = {'coordinates': [choice_start.value, choice_dest.value], 
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': False,
                 'options': {'avoid_polygons': mapping(union_buffer)}} 
route_diesel = clnt.directions(**diesel_request)

# Build popup
duration, distance = route_diesel['features'][0]['properties']['summary'][0].values()
popup = folium.Popup(popup_route.format('Diesel Route',
                                                 duration/60,
                                                 distance/1000))

folium.GeoJson(route_diesel,
               style_function=style_function('#009900'), 
               name='Route after Jan 2019').add_child(popup).add_to(map_berlin)

map_berlin.add_child(folium.map.LayerControl())

If your route will be affected by the diesel ban the duration as well as the distance will extend. Click on each route and check out for more information. With the layer control in the top right corner you can turn on and off the different routes.